## 1. SageMaker Studio 에서 Pipeline 실행

지금 region에서 SageMaker Studio를 실행합니다. Event Engine을 사용하시는 분들은 이미 SageMaker Studio가 생성되어 있기 때문에 AWS 콘솔에서 SageMaker에 있는 Studio를 클릭하여 **open Studio**를 클릭하여 환경으로 접속하게 됩니다.

<p align="center">
<center><img src="./img/studio_notebook.png" height="250" width="850" alt=""><center>
<br><br>
</p>
    
SageMaker studio에 접속한 다음, SageMaker의 Pipeline을 생성하기 위해 Project를 생성합니다. **create project**를 선택한 다음 **MLOps template for model building and training**을 선택한 다음 **select project template**을 생성합니다.
<p align="center">
<center><img src="./img/project_create.png" height="250" width="850" alt=""><center>
<br><br>
</p>   
    
원하시는 project 이름을 입력한 다음, **create project**를 클릭하여 project를 생성합니다.
<p align="center">
<center><img src="./img/project_create_detail.png" height="150" width="550" alt=""><center>
<br><br>
</p>   

In [3]:
import boto3
import json
from sagemaker import get_execution_role
from time import strftime
import calendar

In [4]:
%store -r
print(f"default_bucket : {default_bucket}")

default_bucket : sagemaker-us-west-2-653762495386


### 1. 활용할 소스코드를 S3로 업로드 하기

데이터 과학자가 개발한 소스코드를 S3 또는 CodeCommit, Github 등을 통해 플랫폼 엔지니어가 개발하는 플랫폼으로 전달할 수 있습니다. 여기에서는 간단한 HOL 구성을 위해 코드 레포지토리 대신 S3에 저장하여 옮기는 것으로 구현하였습니다.

In [97]:
!aws s3 sync ./Informer2020/ s3://$default_bucket/Informer2020 --quiet

In [72]:
iam_client = boto3.client('iam')
role=get_execution_role()
base_role_name=role.split('/')[-1]

In [73]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()['Account']

### 2. MLOps에서 활용할 Policy 설정하기

해당 HOL에서 구현할 아키텍처에 필요한 managed policy를 아래와 같이 정의합니다. Role을 별도 생성하셔도 되지만 HOL의 편의성을 위해 SageMaker Notebook/Studio와 동일한 Role에 policy를 추가하여 계속 활용합니다.

In [74]:
# qs_s3_policy=json.dumps({
#     "Version": "2012-10-17",
#     "Statement": [
#         {
#             "Effect": "Allow",
#             "Action": "s3:ListAllMyBuckets",
#             "Resource": "arn:aws:s3:::*"
#         },
#         {
#             "Action": [
#                 "s3:ListBucket",
#                 "s3:ListBucketMultipartUploads",
#                 "s3:GetBucketLocation"
#             ],
#             "Effect": "Allow",
#             "Resource": [
#                 "arn:aws:s3:::sagemaker-us-west-2-322537213286",
#                 "arn:aws:s3:::sagemaker-us-east-2-322537213286"
#             ]
#         },
#         {
#             "Action": [
#                 "s3:GetObject",
#                 "s3:GetObjectVersion",
#                 "s3:PutObject",
#                 "s3:AbortMultipartUpload",
#                 "s3:ListMultipartUploadParts"
#             ],
#             "Effect": "Allow",
#             "Resource": [
#                 "arn:aws:s3:::sagemaker-us-west-2-322537213286/*",
#                 "arn:aws:s3:::sagemaker-us-east-2-322537213286/*"
#             ]
#         }
#     ]
# })

In [75]:
# response = iam_client.create_policy(
#     PolicyName='AWSQuickSightS3Policy',
#     Path='service-role/',
#     PolicyDocument=qs_s3_policy
# )

In [76]:
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'
)
iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSLambda_FullAccess'
)
# iam_client.attach_role_policy(
#     RoleName=base_role_name,
#     PolicyArn='arn:aws:iam::aws:policy/service-role/AWSQuickSightListIAM'
# )
# iam_client.attach_role_policy(
#     RoleName=base_role_name,
#     PolicyArn='arn:aws:iam::322537213286:policy/service-role/AWSQuickSightS3Policy'
# )


{'ResponseMetadata': {'RequestId': '40319253-c6b3-4650-b926-6f505b426836',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '40319253-c6b3-4650-b926-6f505b426836',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 06 Oct 2021 13:29:43 GMT'},
  'RetryAttempts': 0}}

In [77]:
sm_client = boto3.client("sagemaker")

### 3. SageMaker Studio의 설정값에서 model_package_group_name 가져오기

In [78]:
project_name=sm_client.list_projects(
    SortBy='CreationTime',
    SortOrder='Descending')['ProjectSummaryList'][0]['ProjectName']
project_name

'informer-test'

In [79]:
project_response=sm_client.describe_project(ProjectName=project_name)

In [80]:
model_package_group_name = project_response['ProjectName']+"-"+project_response['ProjectId']

### 4. Create Amazon EventBridge Rule

model registry에서 모델이 **Approved**되었을 때 이벤트 트리거를 만들기 위한 설정을 Amazon EventBridge Rule을 이용하여 설정합니다.

In [81]:
event_client = boto3.client('events')

In [82]:
eventpattern = json.dumps(
    {
      "source": ["aws.sagemaker"],
      "detail-type": ["SageMaker Model Package State Change"],
      "detail": {
        "ModelPackageGroupName": [f"{model_package_group_name}"],
        "ModelApprovalStatus": ["Approved"]
      }
    }
)

In [83]:
rule_name = 'informer_model_package_state'
event_rule = event_client.put_rule(
    Name=rule_name,
    EventPattern=eventpattern,
    State='ENABLED',
    Description='This is after the approval update for the Informer model',
)

### 5. Lambda function 생성

EventBridge 에서 Rule 만족하는 이벤트가 발생했을 때 실행되는 Lambda Function을 정의합니다.

In [84]:
code_location = f"s3://{default_bucket}/sagemaker_lambda/"
zip_filename = 'lambda.zip'

In [85]:
%%bash
cd ./Informer2020/sagemaker_lambda/
zip lambda.zip create_model.py

updating: create_model.py (deflated 64%)


In [86]:
!aws s3 cp ./Informer2020/sagemaker_lambda/$zip_filename $code_location

upload: Informer2020/sagemaker_lambda/lambda.zip to s3://sagemaker-us-west-2-653762495386/sagemaker_lambda/lambda.zip


In [87]:
lambda_client = boto3.client('lambda')

In [88]:
lambda_trust_policy=json.dumps({
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
})

In [89]:
role_name='lambda-assume-role'
try:
    iam_client.detach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.detach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )
    iam_client.delete_role(RoleName=role_name)
except:
    pass
finally:
    lambda_role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=lambda_trust_policy
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
    )

KMS 에러가 발생한 경우 lambda_function을 다시 생성합니다.

In [90]:
lambda_name='informer-model-creation-lambda-' + strftime("%m%d-%H%M%s")
try:
    lambda_client.delete_function(FunctionName=lambda_name)
except:
    pass
finally:
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_name,
        Runtime='python3.9',
        Role=lambda_role['Role']['Arn'],
        Handler='create_model.lambda_handler',
        Code={
            'S3Bucket': f'{default_bucket}',
            'S3Key': f'sagemaker_lambda/{zip_filename}'
        },
        Description='Create the latest version-based Informer model',
        Timeout=600,
        MemorySize=256,
        Environment={
          'Variables': {
              "role" : role,
              "default_bucket" : default_bucket
          }
      }
    )

In [91]:
lambda_permission_response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='InvokeLambdaFunction',
    Action='lambda:InvokeFunction',
    Principal="events.amazonaws.com",
    SourceArn=event_rule['RuleArn'],
)

Amazon EventBridge에 위에서 생성한 Lambda function을 타켓으로 설정합니다.

In [92]:
event_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            'Id': 'Target0',
            'Arn': lambda_response['FunctionArn']
        }
    ]
)

{'FailedEntryCount': 0,
 'FailedEntries': [],
 'ResponseMetadata': {'RequestId': 'b352d23e-4db0-4cec-bb06-e0d61a5a292b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b352d23e-4db0-4cec-bb06-e0d61a5a292b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '41',
   'date': 'Wed, 06 Oct 2021 13:29:53 GMT'},
  'RetryAttempts': 0}}

### 6. Studio에서 실행하기

Project가 생성된 다음 아래와 같이 구성된 환경에서 코드를 가져올 수 있습니다. 아래에서 clone repository를 클릭하여 자신의 SageMaker Studio 환경으로 코드를 가져옵니다.
<p align="center">
<center><img src="./img/git_clone.png" height="250" width="750" alt=""><center>
<br><br>
</p>   

가져온 코드 리스트 중에서 .ipynb로 끝나는 노트북 파일을 하나 선택하여 클릭한 다음, **오른쪽 상단의 Kernel**을 클릭하여 아래와 같이 나오는 팝업 창에서 **Python 3(Data Science)**를 선택합니다. 
<p align="center">
<center><img src="./img/mlops_code.png" height="250" width="750" alt=""><center>
<br><br>
</p>  
    

아래 값은 해당 노트북이 아니라 위에서 생성한 SageMaker Studio에서 수행이 필요합니다.
SageMaker Studio의 노트북에서 아래 값을 그대로 copy하여 실행을 합니다.    
<p align="center">
<center><img src="./img/notebook_exe.png" height="250" width="750" alt=""><center>
<br><br>
</p>  

In [98]:
print(f"!aws s3 sync s3://{default_bucket}/Informer2020 ./Informer2020")

!aws s3 sync s3://sagemaker-us-west-2-653762495386/Informer2020 ./Informer2020


이후 SageMaker Studio에서 모든 소스코드를 commit한 다음 push를 하면 이후 자동으로 SageMaker Pipeline은 실행이 됩니다. 실행되는 모습은 SageMaker Pipeline에서 확인하거나 Codepipeline에서 확인하실 수 있습니다.
<p align="center">
<center><img src="./img/code_commit.png" height="250" width="750" alt=""><center>
<br><br>
</p>  
<p align="center">
<center><img src="./img/mlops_exe.png" height="350" width="750" alt=""><center>
<br><br>
</p>  

### 7. QuickSight에서 사용할 manifest_file 생성하기

quicksight에서 사용할 manifest_file 생성합니다.

In [94]:
manifest_file = {
                "fileLocations": [
                    {
                        "URIPrefixes": [
                            f"s3://{default_bucket}/prediciton_result/"
                        ]
                    }
                ],
                "globalUploadSettings": {
                    "format": "CSV",
                    "delimiter": ",",
                    "textqualifier": "\"",
                    "containsHeader": "true"
                }
            }

In [95]:
with open("./quicksight/manifest_file.json", 'w', encoding="utf-8") as f:
    json.dump(manifest_file, f, indent="\t")

In [96]:
!aws s3 cp ./quicksight/manifest_file.json s3://$default_bucket/quick_sight/

upload: quicksight/manifest_file.json to s3://sagemaker-us-west-2-653762495386/quick_sight/manifest_file.json


### refresh-quicksight spice dataset

In [ ]:
qs_client = boto3.client("quicksight")

In [ ]:
datasets_ids = ['f48083af-f596-4101-adfe-dfc07957f7ff']

In [ ]:
import uuid

In [ ]:
res = qs_client.list_data_sets(AwsAccountId=account_id)

# filter out your datasets using a prefix. All my datasets have chicago_crimes as their prefix
datasets_ids = [summary["DataSetId"] for summary in res["DataSetSummaries"]]
ingestion_ids = []

for dataset_id in datasets_ids:
    try:
        ingestion_id = str(calendar.timegm(time.gmtime()))
#         ingestion_id = str(uuid.uuid4())
        qs_client.create_ingestion(DataSetId=dataset_id, IngestionId=ingestion_id,
                                             AwsAccountId=account_id)
        ingestion_ids.append(ingestion_id)
    except Exception as e:
        print(e)
        pass

for ingestion_id, dataset_id in zip(ingestion_ids, datasets_ids):
    while True:
        response = qs_client.describe_ingestion(DataSetId=dataset_id,
                                             IngestionId=ingestion_id,
                                             AwsAccountId=account_id)
        if response['Ingestion']['IngestionStatus'] in ('INITIALIZED', 'QUEUED', 'RUNNING'):
            time.sleep(5)     #change sleep time according to your dataset size
        elif response['Ingestion']['IngestionStatus'] == 'COMPLETED':
            print("refresh completed. RowsIngested {0}, RowsDropped {1}, IngestionTimeInSeconds {2}, IngestionSizeInBytes {3}".format(
                response['Ingestion']['RowInfo']['RowsIngested'],
                response['Ingestion']['RowInfo']['RowsDropped'],
                response['Ingestion']['IngestionTimeInSeconds'],
                response['Ingestion']['IngestionSizeInBytes']))
            break
        else:
            print("refresh failed for {0}! - status {1}".format(dataset_id, response['Ingestion']['IngestionStatus']))
            break